In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

# Prepare dataset

In [2]:
# Load all gbtc data
df_minute_gbtc_2020_12_01_2021_04_18 = pd.read_csv("../data/raw/gbtc_minute_2020-12-01_2021-04-18.csv", sep=",", index_col='date')
df_minute_gbtc_2021_04_18_2021_05_01 = pd.read_csv("../data/raw/gbtc_minute_2021-04-18_2021-05-01.csv", sep=",", index_col='date')
df_minute_gbtc_2020_12_01_2021_04_18.index.astype('datetime64[ns]')
df_minute_gbtc_2021_04_18_2021_05_01.index.astype('datetime64[ns]')

df_gbtc_minute = pd.concat([df_minute_gbtc_2020_12_01_2021_04_18, df_minute_gbtc_2021_04_18_2021_05_01]).drop_duplicates().sort_index(ascending=True)
df_gbtc_minute['time'] = (df_gbtc_minute['datetime'] / 1000).astype('int64')
# df_min = pd.concat([df_minute_gbtc_2020_12_01_2021_04_18, {more data in future}]).drop_duplicates().sort_index(ascending=True)

df_futu_minute_gbtc_2020_06_01_2021_05_06 = pd.read_csv("../data/raw/analysis/data/raw/futu_gbtc_minute_2020-06-01_2021-05-06.csv", sep=",", index_col='date')

# Load all btc data
df_minute_btc_usd_minute_2020_06_01_2021_04_18 = pd.read_csv("../data/raw/btc-usd_minute_2020-06-01_2021-04-18.csv", sep=",", index_col='date')
df_minute_btc_usd_minute_2020_06_01_2021_04_18.index.astype('datetime64[ns]')
df_minute_btc_usd_minute_2021_04_18_2021_05_01 = pd.read_csv("../data/raw/btc-usd_minute_2021-04-18_2021-05-01.csv", sep=",", index_col='date')
df_minute_btc_usd_minute_2021_04_18_2021_05_01.index.astype('datetime64[ns]')

df_btc_minute = pd.concat([df_minute_btc_usd_minute_2020_06_01_2021_04_18, df_minute_btc_usd_minute_2021_04_18_2021_05_01]).drop_duplicates().sort_index(ascending=True)

df_btc_minute = df_btc_minute.sort_index(ascending=True)
df_btc_minute = df_btc_minute['2021-03-01': ]



In [3]:
# Join btc with gbtc on time.
df_minute = pd.merge(df_btc_minute, df_gbtc_minute, on="time", how="left")

df_minute = df_minute.rename(columns = {'low_price':'btc_low_price', 'high_price':'btc_high_price', 'close_price':'btc_close_price', 'open_price':'btc_open_price', 'volume_x':'btc_volume', 'time':'time', 'open':'gbtc_open_price', 'high':'gbtc_high_price', 
'low':'gbtc_low_price', 'close':'gbtc_close_price', 'volume_y':'gbtc_volume'})

df_minute = df_minute.drop(columns=['datetime'])
df_minute['datetime'] = df_minute['time'].apply(lambda d: datetime.utcfromtimestamp(d)) 
df_minute = df_minute.set_index('datetime')
df_minute.index.astype('datetime64[ns]')

btc_per_share = (0.00094509 + 0.000944257) / 2 

df_minute["nav_open_price"] = df_minute["btc_open_price"] * btc_per_share
df_minute["nav_close_price"] = df_minute["btc_close_price"] * btc_per_share
df_minute["nav_high_price"] = df_minute["btc_high_price"] * btc_per_share
df_minute["nav_low_price"] = df_minute["btc_low_price"] * btc_per_share

df_minute["premium"] = (df_minute["gbtc_close_price"] - df_minute["nav_close_price"]) / df_minute["nav_close_price"]
df_minute["premium_high"] = (df_minute["gbtc_high_price"] - df_minute["nav_high_price"]) / df_minute["nav_high_price"]
df_minute["premium_open"] = (df_minute["gbtc_open_price"] - df_minute["nav_open_price"]) / df_minute["nav_open_price"]
df_minute["premium_close"] = (df_minute["gbtc_close_price"] - df_minute["nav_close_price"]) / df_minute["nav_close_price"]
df_minute["premium_low"] = (df_minute["gbtc_low_price"] - df_minute["nav_low_price"]) / df_minute["nav_low_price"]
# df_minute = df_minute.dropna()
df_minute.to_csv("../data/btc_gbtc/btc_gbtc_minute_2021.csv")

In [40]:
# Add extra information
from ta.trend import SMAIndicator, EMAIndicator, ADXIndicator, MACD
from ta.momentum import RSIIndicator

# 1) preimum Simple moving average 
df_minute_pure_gbtc = df_minute.drop_duplicates().dropna()
indicator_sma_15 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=15, fillna=True)
premium_sma15 = indicator_sma_15.sma_indicator().dropna().to_frame()
df_indicators = df_minute.join(premium_sma15)

indicator_sma_30 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=30, fillna=True)
premium_sma30 = indicator_sma_30.sma_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_sma30)

indicator_sma_60 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=60, fillna=True)
premium_sma60 = indicator_sma_60.sma_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_sma60)

indicator_sma_120 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=120, fillna=True)
premium_sma120 = indicator_sma_120.sma_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_sma120)

indicator_sma_180 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=180, fillna=True)
premium_sma180 = indicator_sma_180.sma_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_sma180)

indicator_sma_240 = SMAIndicator(close=df_minute_pure_gbtc["premium"], window=240, fillna=True)
premium_sma240 = indicator_sma_240.sma_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_sma240)

# 2) Premium Exponential moving average
indicator_ema_15 = EMAIndicator(close=df_minute_pure_gbtc["premium"], window=15, fillna=True)
premium_ema15 = indicator_ema_15.ema_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_ema15)

indicator_ema_30 = EMAIndicator(close=df_minute_pure_gbtc["premium"], window=30, fillna=True)
premium_ema30 = indicator_ema_30.ema_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_ema30)

indicator_ema_60 = EMAIndicator(close=df_minute_pure_gbtc["premium"], window=60, fillna=True)
premium_ema60 = indicator_ema_60.ema_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_ema60)

indicator_ema_120 = EMAIndicator(close=df_minute_pure_gbtc["premium"], window=120, fillna=True)
premium_ema120 = indicator_ema_120.ema_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_ema120)

indicator_ema_180 = EMAIndicator(close=df_minute_pure_gbtc["premium"], window=180, fillna=True)
premium_ema180 = indicator_ema_180.ema_indicator().dropna().to_frame()
df_indicators = df_indicators.join(premium_ema180)

# 3) ADX (premium, btc)
premium_adx_indicator = ADXIndicator(high=df_minute_pure_gbtc["premium_high"], low=df_minute_pure_gbtc["premium_low"], close=df_minute_pure_gbtc["premium_close"], window=14, fillna=False)
premium_adx = premium_adx_indicator.adx().dropna().to_frame() 
premium_adx = premium_adx.rename(columns={'adx':'premium_adx'})
premium_adx_neg = premium_adx_indicator.adx_neg().dropna().to_frame() 
premium_adx_neg = premium_adx_neg.rename(columns={'adx_neg':'premium_adx_neg'})
premium_adx_pos = premium_adx_indicator.adx_pos().dropna().to_frame()
premium_adx_pos = premium_adx_pos.rename(columns={'adx_pos':'premium_adx_pos'})

# btc_adx_indicator = ADXIndicator(high=df_minute["btc_high_price"], low=df_minute["btc_low_price"], close=df_minute["btc_close_price"], window=14, fillna=True)
df_indicators = df_indicators.join(premium_adx)
df_indicators = df_indicators.join(premium_adx_neg)
df_indicators = df_indicators.join(premium_adx_pos)

btc_adx_indicator = ADXIndicator(high=df_minute["btc_open_price"], low=df_minute["btc_low_price"], close=df_minute["btc_close_price"], window=14, fillna=True)
btc_adx = btc_adx_indicator.adx().dropna().to_frame() 
btc_adx = btc_adx.rename(columns={'adx':'btc_adx'})
btc_adx_neg = btc_adx_indicator.adx_neg().dropna().to_frame() 
btc_adx_neg = btc_adx_neg.rename(columns={'adx_neg':'btc_adx_neg'})
btc_adx_pos = btc_adx_indicator.adx_pos().dropna().to_frame() 
btc_adx_pos = btc_adx_pos.rename(columns={'adx_pos':'btc_adx_pos'})

df_indicators = df_indicators.join(btc_adx)
df_indicators = df_indicators.join(btc_adx_neg)
df_indicators = df_indicators.join(btc_adx_pos)

# 4) MACD (premium, btc)
premium_macd_indicator = MACD(close=df_minute_pure_gbtc["premium_close"], window_slow=26, window_fast=12, window_sign=9, fillna=True)
premium_macd_line = premium_macd_indicator.macd().dropna().to_frame() 
premium_macd_line = premium_macd_line.rename(columns={'MACD_12_26':'premium_macd_12_26'})
premium_macd_histo = premium_macd_indicator.macd_diff().dropna().to_frame() 
premium_macd_histo = premium_macd_histo.rename(columns={'MACD_diff_12_26':'premium_macd_histo_12_26'})
premium_macd_signal = premium_macd_indicator.macd_signal().dropna().to_frame() 
premium_macd_signal = premium_macd_signal.rename(columns={'MACD_sign_12_26':'premium_macd_sign_12_26'})

df_indicators = df_indicators.join(premium_macd_line)
df_indicators = df_indicators.join(premium_macd_histo)
df_indicators = df_indicators.join(premium_macd_signal)

btc_macd_indicator = MACD(close=df_minute["btc_close_price"], window_slow=26, window_fast=12, window_sign=9, fillna=True)
btc_macd_line = btc_macd_indicator.macd().dropna().to_frame() 
btc_macd_line = btc_macd_line.rename(columns={'MACD_12_26':'btc_macd_12_26'})
btc_macd_histo = btc_macd_indicator.macd_diff().dropna().to_frame() 
btc_macd_histo = btc_macd_histo.rename(columns={'MACD_diff_12_26':'btc_macd_histo_12_26'})
btc_macd_signal = btc_macd_indicator.macd_signal().dropna().to_frame() 
btc_macd_signal = btc_macd_signal.rename(columns={'MACD_sign_12_26':'btc_macd_sign_12_26'})

df_indicators = df_indicators.join(btc_macd_line)
df_indicators = df_indicators.join(btc_macd_histo)
df_indicators = df_indicators.join(btc_macd_signal)

# 5) SCI (btc)
btc_rsi_indicator = RSIIndicator(close=df_minute["btc_close_price"], window = 14, fillna = True)
btc_rsi = btc_rsi_indicator.rsi()
df_indicators = df_indicators.join(btc_rsi)

df_indicators.to_csv("../data/btc_gbtc/btc_gbtc_minute_with_indicators_2021.csv")


/Users/chi.wang/miniconda3/envs/robinhood/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/Users/chi.wang/miniconda3/envs/robinhood/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars

/Users/chi.wang/miniconda3/envs/robinhood/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/Users/chi.wang/miniconda3/envs/robinhood/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars



In [43]:
import datetime
# theday = datetime.date.today()
# weekday = theday.isoweekday()
# # The start of the week
# start = theday - datetime.timedelta(days=weekday)
# # build a simple range
# dates = [start + datetime.timedelta(days=d) for d in range(7)]

# # Starts with knowing the day of the week
# week_day=datetime.datetime.now().isocalendar()[2]

# # Calculates Starting date (Sunday) for this case by subtracting current date with time delta of the day of the week
# start_date=datetime.datetime.now() - datetime.timedelta(days=week_day)

# # Prints the list of dates in a current week
# dates=[str((start_date + datetime.timedelta(days=i)).date()) for i in range(7)]
# dates

In [44]:
# divide data to weeks
weeks = [ ("2021-03-03", "2021-03-08"), ("2021-03-08", "2021-03-15"), ("2021-03-15", "2021-03-22"), ("2021-03-22", "2021-03-29"), ("2021-03-29", "2021-04-04"), ("2021-04-04", "2021-04-12"), ("2021-04-12", "2021-04-19"), ("2021-04-19", "2021-04-24"), ("2021-04-24", "2021-04-30")]

df_indicators.index[0]
start_date = weeks[0][0]
end_date = weeks[0][1]
mask = (df_indicators.index > start_date) & (df_indicators.index <= end_date)
df = df_indicators.loc[mask]

# df

# Graph

In [50]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# subplots example
# https://plotly.com/python/subplots/

# Multiple axes and subplots
# https://maegul.gitbooks.io/resguides-plotly/content/content/plotting_locally_and_offline/python/multiple_axes_and_subplots.html
# https://plotly.com/python/subplots/#stacked-subplots-with-a-shared-xaxis-lowlevel-api (This is what I followed)

class IndicatorType:
    ADX_PREMIUM = 1
    ADX_BTC = 2
    MACD_PREMIUM = 3
    MACD_BTC = 4
    RSI = 5

def plot_price_with_premium_chart(df, chart_width=5000, chart_height=800, nticks=200, ema=False, indicator_type=IndicatorType.ADX_PREMIUM):
    
    # 1) define data points
    nav_data = go.Scatter(x=df.index, y=df["nav_close_price"], mode='lines+markers', name="btc nav close price")
    gbtc_data = go.Scatter(x=df.index, y=df["gbtc_close_price"], mode='lines+markers', name="gbtc close price")
    premium_data = go.Scatter(x=df.index, y=df["premium"], mode='lines+markers', name="premium percentage", yaxis='y2')
    data = [nav_data, gbtc_data, premium_data]

    # EMA on minutes
    premium_ema15 = go.Scatter(x=df.index, y=df["ema_15"], mode='lines+markers', name="premium 15 EMA", yaxis='y2')
    premium_ema30 = go.Scatter(x=df.index, y=df["ema_30"], mode='lines+markers', name="premium 30 EMA", yaxis='y2')
    premium_ema60 = go.Scatter(x=df.index, y=df["ema_60"], mode='lines+markers', name="premium 60 EMA", yaxis='y2')
    premium_ema120 = go.Scatter(x=df.index, y=df["ema_120"], mode='lines+markers', name="premium 120 EMA", yaxis='y2')
    premium_ema180 = go.Scatter(x=df.index, y=df["ema_180"], mode='lines+markers', name="premium 180 EMA", yaxis='y2')
    
    # SMA on minutes
    premium_sema15 = go.Scatter(x=df.index, y=df["sma_15"], mode='lines+markers', name="premium 15 SMA", yaxis='y2')
    premium_sma30 = go.Scatter(x=df.index, y=df["sma_30"], mode='lines+markers', name="premium 30 SMA", yaxis='y2')
    premium_sma60 = go.Scatter(x=df.index, y=df["sma_60"], mode='lines+markers', name="premium 60 SMA", yaxis='y2')
    premium_sma120 = go.Scatter(x=df.index, y=df["sma_120"], mode='lines+markers', name="premium 120 SMA", yaxis='y2')
    premium_sma180 = go.Scatter(x=df.index, y=df["sma_180"], mode='lines+markers', name="premium 180 SMA", yaxis='y2')
    premium_sma240 = go.Scatter(x=df.index, y=df["sma_240"], mode='lines+markers', name="premium 240 SMA", yaxis='y2')

    if ema: 
        data.append(premium_ema180)
    else:
        # data.append(premium_sma180)
        data.append(premium_sma240)

    premium_adx = go.Scatter(x=df.index, y=df["premium_adx"], mode='lines+markers', name="premium adx", yaxis='y3')
    premium_adx_neg = go.Scatter(x=df.index, y=df["premium_adx_neg"], mode='lines+markers', name="premium -DI", yaxis='y3')
    premium_adx_pos = go.Scatter(x=df.index, y=df["premium_adx_pos"], mode='lines+markers', name="premium +DI", yaxis='y3')

    btc_adx = go.Scatter(x=df.index, y=df["btc_adx"], mode='lines+markers', name="btc adx", yaxis='y3')
    btc_adx_neg = go.Scatter(x=df.index, y=df["btc_adx_neg"], mode='lines+markers', name="btc -DI", yaxis='y3')
    btc_adx_pos = go.Scatter(x=df.index, y=df["btc_adx_pos"], mode='lines+markers', name="btc +DI", yaxis='y3')

    btc_rsi = go.Scatter(x=df.index, y=df["rsi"], mode='lines+markers', name="premium adx", yaxis='y3')

    btc_macd = go.Scatter(x=df.index, y=df["btc_macd_12_26"], mode='lines+markers', name="btc macd", yaxis='y3')
    btc_macd_sign = go.Scatter(x=df.index, y=df["btc_macd_sign_12_26"], mode='lines+markers', name="btc macd sign", yaxis='y3')
    btc_macd_histo = go.Scatter(x=df.index, y=df["btc_macd_histo_12_26"], mode='lines+markers', name="btc macd histo", yaxis='y3')

    if indicator_type == IndicatorType.ADX_PREMIUM:
        data.append(premium_adx)
        # data.append(premium_adx_neg)
        # data.append(premium_adx_pos)
        y3 = go.YAxis(title= 'Premium ADX', titlefont=go.Font(color='Red'), domain=[0, 0.3])

    elif indicator_type == IndicatorType.ADX_BTC:
        data.append(btc_adx)
        data.append(btc_adx_neg)
        data.append(btc_adx_pos)

        y3 = go.YAxis(title= 'BTC ADX', titlefont=go.Font(color='Red'), domain=[0, 0.3])
    elif indicator_type == IndicatorType.RSI:
        data.append(btc_rsi)
        y3 = go.YAxis(title= 'BTC RSI', titlefont=go.Font(color='Red'), domain=[0, 0.3])
    elif indicator_type == IndicatorType.MACD_BTC:
        data.append(btc_macd)
        data.append(btc_macd_sign)
        data.append(btc_macd_histo)
        y3 = go.YAxis(title= 'BTC MACD', titlefont=go.Font(color='Red'), domain=[0, 0.3])

    y1 = go.YAxis(title='Actual Price', titlefont=go.Font(color='SteelBlue'), domain=[0.35, 1])
    y2 = go.YAxis(title= 'Premium', titlefont=go.Font(color='DarkOrange'), domain=[0.35, 1])
    y2.update(overlaying='y', side='right')
    
    layout = go.Layout(yaxis1 = y1, yaxis2 = y2, yaxis3 = y3, width=chart_width, height=chart_height)
    fig = go.Figure(data=data, layout=layout)
    fig.update_xaxes(nticks=nticks)  
    # fig.add_hline(y=35, row=1, line_color="green", annotation_text="ADX threshold", annotation_position="bottom left")

    fig.show()



In [51]:
# total 7 weeks since Mar 3, try increase the week_no for other week's data
week_no = 2
start_date = weeks[week_no][0]
end_date = weeks[week_no][1]
mask = (df_indicators.index > start_date) & (df_indicators.index <= end_date)
df = df_indicators.loc[mask]

# plot premium and its average 
# plot_price_with_premium_chart(df, chart_width=5000, indicator_type=IndicatorType.MACD_BTC)
# plot_price_with_premium_chart(df, chart_width=5000, ema = True)


plot_price_with_premium_chart(df, chart_width=5000, chart_height=1200, indicator_type=IndicatorType.ADX_PREMIUM)